In [1]:
import os
import sys
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline
#os.chdir('D:/OBS/GeoData')
#sns.reset_orig

<a id='0'></a>
# Outline
- Generate Flow map. 

## [Part 0 Functions](#0)

## [Part 1 Process](#1)

<a id='0.0'></a>
## [Part 0 Functions](#0)

In [2]:
def get_df_max(df):
    max_value = 0
    for item in df.columns:
        if df[item].max() > max_value:
            max_value = df[item].max()
    return max_value

def point_generator(shape):
    
    x_lon = np.zeros((len(shape),1))
    y_lat = np.zeros((len(shape),1))
    for ip in range(len(shape)):
        x_lon[ip] = shape[ip][0]
        y_lat[ip] = shape[ip][1]
    return x_lon, y_lat

def flow_z_score(z_score):
    if z_score <= -4:
        return 1
    elif z_score > -4 and z_score <= -3:
        return 2
    elif z_score > -3 and z_score <= -2:
        return 3
    elif z_score > -2 and z_score <= -1:
        return 4
    elif z_score > -1 and z_score < 0:
        return 5
    elif z_score >= 0 and z_score < 1:
        return 6
    elif z_score >= 1 and z_score < 2:
        return 7
    elif z_score >= 2 and z_score < 3:
        return 8
    elif z_score >= 3 and z_score < 4:
        return 9
    elif z_score >= 4:
        return 10

In [3]:
def plot_map_2(id ,sf, normal_color, line_color, line_width, ax_input, x_lim = None, y_lim = None):
    main_fill = '#b2182b'
    sub_fill = normal_color
    
    count = 0
    for shape in list(sf.iterShapes()):
        #sys.stdout.write('\rprocessing poliygon {}'.format(count+1))
        nparts = len(shape.parts) # total parts
        npoints=len(shape.points) # total points
        
        if nparts == 1:                
            points = point_generator(shape.points)
            if count == id:
                ax_input.plot(points[0],points[1], line_color, zorder=10, linewidth=line_width)
                ax_input.fill(points[0],points[1], main_fill, zorder=10)
            else:
                ax_input.plot(points[0],points[1], line_color, linewidth=line_width)
                ax_input.fill(points[0],points[1], sub_fill)
            count += 1

        else:
            if count == id:
                for ip in range(nparts): # loop over parts, plot separately
                    i0=shape.parts[ip]
                    if ip < nparts-1:
                        i1 = shape.parts[ip+1]-1
                    else:
                        i1 = npoints

                    seg=shape.points[i0:i1+1]              
                    points = point_generator(seg)
                    ax_input.plot(points[0],points[1], line_color, zorder=10, linewidth=line_width)
                    ax_input.fill(points[0],points[1], main_fill, zorder=10)
            else:
                for ip in range(nparts): # loop over parts, plot separately
                    i0=shape.parts[ip]
                    if ip < nparts-1:
                        i1 = shape.parts[ip+1]-1
                    else:
                        i1 = npoints

                    seg=shape.points[i0:i1+1]
                    points = point_generator(seg)
                    ax_input.plot(points[0],points[1], line_color, linewidth=line_width)
                    ax_input.fill(points[0],points[1], sub_fill)
            count += 1
            
    if (x_lim != None) & (y_lim != None):     
        ax_input.set_xlim(x_lim)
        ax_input.set_ylim(y_lim)
        
def plot_map_heat_2(sf, df, weekday, hour, line_color, line_width, 
                    ax_input, saved_folder = None, x_lim = None, y_lim = None):
    color_map = {
        1:'#313695',2:'#4575b4',3:'#74add1',4:'#abd9e9',5:'#e0f3f8',
        6:'#fee090',7:'#fdae61',8:'#f46d43',9:'#d73027',10:'#a50026',
    }
    
    max_row = df[(df['weekday']==weekday) & (df['time']==hour)] \
                .sort_values(by='net_flow',ascending=False).iloc[0]
    min_row = df[(df['weekday']==weekday) & (df['time']==hour)] \
                .sort_values(by='net_flow',ascending=False).iloc[-1]
    
    max_id = max_row['poly_code']
    min_id = min_row['poly_code']
    
    count = 0
    for shape in list(sf.iterShapes()):
        
        nparts = len(shape.parts) # total parts
        npoints= len(shape.points) # total points
        
        if nparts == 1:                
            points = point_generator(shape.points)
            
            color = color_map[flow_z_score\
                              (float\
                               (df['z_score Distribution']\
                                [(df['poly_code']==count) & (df['time'] == hour) & (df['weekday'] == weekday)]))]           
            ax_input.plot(points[0],points[1], line_color, zorder=10, linewidth=line_width)
            
            #Draw green edge flow data of the polygon is either maximum or minimum.
            if count == max_id:
                ax_input.plot(points[0],points[1], '#3aff03', zorder=15, linewidth=2)
            elif count == min_id:
                ax_input.plot(points[0],points[1], '#3aff03', zorder=15, linewidth=2)
            
            
            ax_input.fill(points[0],points[1], color, zorder=10)
        
            count += 1

        else:
            for ip in range(nparts): # loop over parts, plot separately
                i0=shape.parts[ip]
                if ip < nparts-1:
                    i1 = shape.parts[ip+1]-1
                else:
                    i1 = npoints

                seg=shape.points[i0:i1+1]              
                points = point_generator(seg)
                color = color_map[flow_z_score\
                                  (float\
                                   (df['z_score Distribution']\
                                    [(df['poly_code']==count) & (df['time'] == hour) & (df['weekday'] == weekday)]))]           

                ax_input.plot(points[0],points[1], line_color, zorder=10, linewidth=line_width)
                
                #Draw green edge flow data of the polygon is either maximum or minimum.
                if count == max_id:
                    ax_input.plot(points[0],points[1], '#3aff03', zorder=15, linewidth=2)
                elif count == min_id:
                    ax_input.plot(points[0],points[1], '#3aff03', zorder=15, linewidth=2)
                    
                ax_input.fill(points[0],points[1], color, zorder=10)

            count += 1
    #Zoom the map        
    if (x_lim != None) & (y_lim != None):     
        ax_input.set_xlim(x_lim)
        ax_input.set_ylim(y_lim)
    
    #Create legend
    legend_std = df['net_flow'].describe()['std']
    legend_mean = df['net_flow'].describe()['mean']
    std_list = []
    for i in range(-4, 5):
        std_list.append(int(legend_std*i))
    
    
    patch10 = mpatches.Patch(facecolor=color_map[10], edgecolor='#000000',
                             label='> {}'.format(std_list[8]))
    patch09 = mpatches.Patch(facecolor=color_map[9], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[7],std_list[8]))
    patch08 = mpatches.Patch(facecolor=color_map[8], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[6],std_list[7]))
    patch07 = mpatches.Patch(facecolor=color_map[7], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[5],std_list[6]))
    patch06 = mpatches.Patch(facecolor=color_map[6], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[4],std_list[5]))
    patch05 = mpatches.Patch(facecolor=color_map[5], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[3],std_list[4]))
    patch04 = mpatches.Patch(facecolor=color_map[4], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[2],std_list[3]))
    patch03 = mpatches.Patch(facecolor=color_map[3], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[1],std_list[2]))
    patch02 = mpatches.Patch(facecolor=color_map[2], edgecolor='#000000',
                             label='{} ~ {}'.format(std_list[0],std_list[1]))
    patch01 = mpatches.Patch(facecolor=color_map[1], edgecolor='#000000', 
                             label='{} >'.format(std_list[0]))
    
    ax_input.legend(handles=[patch10,patch09,patch08,patch07,patch06,
                        patch05,patch04,patch03,patch02,patch01],prop={'size': 9},loc='lower right').set_zorder(20)
    
    weekday_dict = {
        0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'
    }
    title = '{} {}:00'.format(weekday_dict[weekday],hour)
    ax_input.text(121.359, 25.229, title, fontsize=15)
    
    

    content_max = 'Max Inward: {} ({})'.format(max_row['entrance_area'], int(max_row['net_flow']))
    content_min = 'Max Outward: {} ({})'.format(min_row['entrance_area'], int(min_row['net_flow']))
    
    ax_input.text(121.359, 24.9245, content_max, fontsize=12, zorder=21)
    ax_input.text(121.359, 24.9095, content_min, fontsize=12, zorder=21)
    
    if saved_folder != None:
        plt.savefig('{}/flow_map_{}_{}.png'.format(saved_folder,weekday,hour),
                    bbox_inches='tight')
    #plt.close()

#Plot MRT station
def plot_point_2(df,ax_input):
    for idx,row in df.iterrows():
        ax_input.plot(row['long'],row['lat'],
                     marker='.',
                     markersize=2,
                     markeredgewidth=1,
                     markeredgecolor='k',
                     color='#525252',
                     zorder=14)

def bar_color(patch_value):
    for i in range(0,10):
        patch_value[i].set_facecolor('#313695')
    for i in range(10,11):
        patch_value[i].set_facecolor('#4575b4')
    for i in range(11,12):
        patch_value[i].set_facecolor('#74add1')
    for i in range(12,13):
        patch_value[i].set_facecolor('#abd9e9')
    for i in range(13,14):
        patch_value[i].set_facecolor('#e0f3f8')
    for i in range(14,15):
        patch_value[i].set_facecolor('#fee090')
    for i in range(15,16):
        patch_value[i].set_facecolor('#fdae61')
    for i in range(16,17):
        patch_value[i].set_facecolor('#f46d43')
    for i in range(17,18):
        patch_value[i].set_facecolor('#d73027')
    for i in range(18,28):
        patch_value[i].set_facecolor('#a50026')
        
    
def movement_map_pack(region1, region2, mrt_station, mrt_dist, df, ax_input, weekday, hour):
    ax_input = plt.axes() # add the axes
    ax_input.set_aspect('equal')
    ax_input.grid(False)
    ax_input.set_xticks([])
    ax_input.set_yticks([])

    y_lim = (24.9,25.25) # latitude 
    x_lim = (121.35, 121.70) # longitude

    plot_map_2('x',region1,'#f0f0f0','#bdbdbd', 0.4, ax_input)
    plot_map_2('x',region2,'#f0f0f0','#d9d9d9', 0.4, ax_input)
    plot_point_2(mrt_station, ax_input)
    plot_map_heat_2(sf=mrt_dist, df=df, weekday=weekday, hour=hour, 
                    line_color='#bdbdbd', line_width=0.4, ax_input=ax_input, 
                    x_lim=x_lim, y_lim=y_lim) 



<a id='1'></a>

## [Part 1 Process](#0)

1. Prepare shapefiles for drawing maps
2. Prepare reference dictionary
3. Read flow data
4. Assign poly_code, ration and Z score
5. Generate maps

In [4]:
#1.1 Prepare shapefiles for drawing maps
regional_path = "geo_data/北北基桃.shp"
district_path = 'geo_data/新北市區界_4.shp'
mrt_district_path = 'geo_data/mrt_district_vor_2.shp'

mrt_route = 'geo_data/TpeMRTRoutes_WGS84_2.shp'
mrt_pnt = 'station_code_with_VorDistrict_2.csv'

region = shp.Reader(regional_path)
new_tpe = shp.Reader(district_path)
mrt_district = shp.Reader(mrt_district_path)
mrt_pnt = pd.read_csv(mrt_pnt)

mrt_district.records()[:5]

[Record #0: ['BR07', 'Liuzhangli', 'A02', 'Daan District', '', '', '', '', '', 'BR07'],
 Record #1: ['G07', 'Gongguan', 'A02', 'Daan District', '', '', '', '', '', 'G07'],
 Record #2: ['BR08', 'Technology Building', 'A02', 'Daan District', '', '', '', '', '', 'BR08'],
 Record #3: ['BR06', 'Linguang', 'A02', 'Daan District', '', '', '', '', '', 'BR06'],
 Record #4: ['R03', 'Taipei 101/World Trade Center', 'A17', 'Xinyi District', '', '', '', '', '', 'R03']]

In [5]:
#1.2 Prepare reference dictionary for labeling data
def record_to_list(record_source):
    data = {}
    id_ = 0
    for item in record_source.records():
        data[item[-1]] = id_
        id_ += 1
    return data

polygon_dict = record_to_list(mrt_district)
polygon_dict

{'BL10': 11,
 'BL11': 21,
 'BL12': 25,
 'BL13': 20,
 'BL14': 22,
 'BL15': 17,
 'BL16': 16,
 'BL17': 15,
 'BL18': 18,
 'BL19': 23,
 'BL20': 26,
 'BR06': 3,
 'BR07': 0,
 'BR08': 2,
 'BR12': 36,
 'BR13': 38,
 'BR14': 42,
 'BR15': 43,
 'BR16': 44,
 'BR17': 41,
 'Banqiao District': 57,
 'Beitou District': 51,
 'Danshui District': 46,
 'G07': 1,
 'G08': 5,
 'G10': 13,
 'G11': 12,
 'G13': 24,
 'G14': 32,
 'G16': 30,
 'G17': 27,
 'G18': 29,
 'G19': 28,
 'Luzhou District': 54,
 'Nangang District': 47,
 'Neihu District': 48,
 'O05': 10,
 'O06': 6,
 'O08': 31,
 'O09': 35,
 'O10': 34,
 'O11': 39,
 'O12': 37,
 'R02': 14,
 'R03': 4,
 'R04': 8,
 'R05': 9,
 'R06': 7,
 'R09': 19,
 'R12': 33,
 'R14': 40,
 'R15': 45,
 'Sanzhong District': 59,
 'Shilin District': 49,
 'Tucheng District': 53,
 'Wenshan District': 55,
 'Xindian District': 52,
 'Xinzhuang District': 58,
 'Yonghe District': 50,
 'Zhonghe District': 56}

In [6]:
#1.3 Read flow data, where polycode is key to connect with correct polygon in shapefile

df = pd.read_csv('district_flow.csv',engine='python')
df['entrance_area'] = df['entrance_area'].str.replace('Dist.', 'District')
df['exit_area'] = df['exit_area'].str.replace('Dist.', 'District')
df['poly_code'] = np.nan
df['ratio'] = np.nan
df.head()

,entrance_area,time,weekday,avg_ppl_in,exit_area,avg_ppl_out,net_flow,poly_code,ratio
0,Banqiao District,18,4,25262.25,Banqiao District,13479.88,11782.37,NaN,NaN
1,Banqiao District,18,1,24634.63,Banqiao District,11635.17,12999.46,NaN,NaN
2,Banqiao District,18,3,24584.21,Banqiao District,11752.91,12831.30,NaN,NaN
3,Banqiao District,18,2,24143.38,Banqiao District,11643.55,12499.83,NaN,NaN
4,Banqiao District,18,0,23770.80,Banqiao District,11268.93,12501.87,NaN,NaN


In [7]:
#1.4 Assign poly_code, ration and Z score

for idx,row in df.iterrows():
    #map polygon id
    for k,v in polygon_dict.items():
        if row[0] == k:
            df.iloc[idx,7] = v
    #map ratio of net income       
    if row[6] >= 0:
        df.iloc[idx,8] = row[6]/df['net_flow'].max()
    else:
        df.iloc[idx,8] = row[6]/df['net_flow'].min()*(-1)   
        
df['z_score Distribution'] = df['net_flow'].apply(lambda x:  (x - df['net_flow'].describe().loc['mean']) / df['net_flow'].describe().loc['std'])
df.head()

,entrance_area,time,weekday,avg_ppl_in,exit_area,avg_ppl_out,net_flow,poly_code,ratio,z_score Distribution
0,Banqiao District,18,4,25262.25,Banqiao District,13479.88,11782.37,57.0,0.906374,7.394496
1,Banqiao District,18,1,24634.63,Banqiao District,11635.17,12999.46,57.0,1.000000,8.158293
2,Banqiao District,18,3,24584.21,Banqiao District,11752.91,12831.30,57.0,0.987064,8.052762
3,Banqiao District,18,2,24143.38,Banqiao District,11643.55,12499.83,57.0,0.961565,7.844745
4,Banqiao District,18,0,23770.80,Banqiao District,11268.93,12501.87,57.0,0.961722,7.846025


In [8]:
#Create folder for keeping maps

dirName = 'flow_graph'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " folder created ")
else:    
    print("Directory " , dirName ,  " already exists")

Directory  flow_graph  already exists


In [9]:
#1.5 Generate maps

weekday_all = np.array(range(0,7)) 
hour_all = [0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

for weekday in weekday_all:
    for time in hour_all:
        sys.stdout.write('\rprocessing weekday: {}, hour: {}'.format(weekday, time))
        f=plt.figure(figsize = (8,12.5))
        gs=GridSpec(6,2)
        ax1=f.add_subplot(gs[:3,:])
        ax1.grid(False)
        ax1.set_xticks([])
        ax1.set_yticks([])
        movement_map_pack(region, new_tpe, mrt_pnt, mrt_district, df, ax1, weekday, time)


        ax2=f.add_subplot(gs[5,:])
        bin_group = np.array(range(-14,15))

        ax2.set_xticks(bin_group[::2])
        ax2.set_ylabel('Counts')
        ax2.set_title('Z Distribution')
        ax2.grid(color='#d9d9d9')
        N, bins, patches = ax2.hist(df['z_score Distribution'][(df['weekday']==weekday) & (df['time']==time)],bins=bin_group)
        bar_color(patches)
        
        f.savefig('{}/map_{}_{}.png'.format(dirName,weekday,time),
                    bbox_inches='tight')
        plt.close()

print('\nDone')
        


processing weekday: 6, hour: 23Done


In [10]:
#Sort by z score
df[(df['weekday']==4) & (df['time']==8)].sort_values(by='z_score Distribution')

,entrance_area,time,weekday,avg_ppl_in,exit_area,avg_ppl_out,net_flow,poly_code,ratio,z_score Distribution
156,Banqiao District,8,4,9029.48,Banqiao District,30297.88,-21268.40,57.0,-0.980886,-13.346828
1689,Zhonghe District,8,4,2670.88,Zhonghe District,17772.85,-15101.97,56.0,-0.696494,-9.477026
1171,Sanzhong District,8,4,3512.28,Sanzhong District,14245.15,-10732.87,59.0,-0.494994,-6.735156
1608,Tucheng District,8,4,2769.33,Tucheng District,10336.85,-7567.52,53.0,-0.349010,-4.748711
1857,Xinzhuang District,8,4,2439.26,Xinzhuang District,9700.59,-7261.33,58.0,-0.334888,-4.556558
1131,Wenshan District,8,4,3587.09,Wenshan District,10239.61,-6652.52,55.0,-0.306810,-4.174494
4098,Yonghe District,8,4,1082.93,Yonghe District,7542.61,-6459.68,50.0,-0.297917,-4.053475
5693,Luzhou District,8,4,551.41,Luzhou District,6692.90,-6141.49,54.0,-0.283242,-3.853792
648,Xindian District,8,4,4891.39,Xindian District,10902.94,-6011.55,52.0,-0.277249,-3.772247
2117,Danshui District,8,4,2184.46,Danshui District,8181.10,-5996.64,46.0,-0.276562,-3.762890


In [11]:
df['net_flow'].describe()  

count     8820.000000
mean        -0.571034
std       1593.474473
min     -21682.840000
25%       -313.552500
50%         -3.605000
75%        266.042500
max      12999.460000
Name: net_flow, dtype: float64